In [3]:
# Import packages
import pandas as pd
import numpy as np
import xml.etree.cElementTree as et
import spacy
import re
from nltk import tokenize
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', None)

/tmp/ipykernel_7758/1898551232.py:8: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


## 1. Extract Metadata: title, publish date, etc.

In [4]:
def extract_meta(path):
    tree=et.parse(path)
    root=tree.getroot()

    meta = {'Keyword': [], 
            'GOID': [], 
            'Title': [], 
            'Contributors': [],
            'Contributor FirstName': [],
            'Contributor LastName': [],
            'Numeric Date': [],
            'Start Date': [],
            'End Date': [],
            'Qualifier': [],
            'Publisher Country': [],
            'Language': [],
            'Source': [],
            'StartPage': [],
            'DocSection': [],
            'ColumnHeader': [],
            'DocEdition': [],
            'Title Keywords': [],
            'Terms': [],
            'Word Count': [],
            'Title of Newspaper': [],
            'Subjects': [],
            'MpubId': []
            }

    # iteratre over the trees to extract metadata
    for item in root.iter('GOID'):
        meta['GOID'].append(item.text)

    for item in root.iter('TitleAtt'):
        meta['Title'].append(item[0].text)

    for item in root.iter('PubFrosting'):
        for title in item.iter('Qualifier'):
            meta['Qualifier'].append(title.text)
            break

    for item in root.iter('ISO'):
        meta['Language'].append(item[1].text.strip())

    # Contributor Info
    for item in root.iter('Contributors'):
        for contributor in item:
            if not meta['Contributors']:
                meta['Contributors'].append(contributor[0][0].text)
            else:
                meta['Contributors'].append('^^'+contributor[0][0].text)

    for item in root.iter('LastName'):
        if not meta['Contributor LastName']:
            meta['Contributor LastName'].append(item.text)
        else:
            meta['Contributor LastName'].append('^^'+item.text)

    for item in root.iter('FirstName'):
        if not meta['Contributor FirstName']:
            meta['Contributor FirstName'].append(item.text)
        else:
            meta['Contributor FirstName'].append('^^'+item.text)

    for item in root.iter('NumericDate'):
        meta['Numeric Date'].append(item.text)

    for item in root.iter('SourceType'):
        meta['Source'].append(item.text)

    for item in root.iter('Country'):
        meta['Publisher Country'].append(item.text)
    
    # PrintLocation Info
    for item in root.iter('StartPage'):
        meta['StartPage'].append(item.text)

    for item in root.iter('DocSection'):
        meta['DocSection'].append(item.text)
    
    for item in root.iter('ColumnHeader'):
        meta['ColumnHeader'].append(item.text)
    
    for item in root.iter('DocEdition'):
        meta['DocEdition'].append(item.text)
    
    for title in root.iter('Terms'):
        terms = [title.tag, title.attrib, title.text.strip()]
        for item in title:
            item_info = [item.tag, item.attrib, item.text.strip()]
            for sub in item:
                item_info.append(sub.tag)
                item_info.append(sub.attrib)
                item_info.append(sub.text)
            terms.append(item_info)
        meta['Terms'].append(terms)

    # for title in root.iter('GenSubjTerm'):
    #     terms = [title.tag, title.attrib, title.text.strip()]
    #     for item in title:
    #         terms.append(item.tag)
    #         terms.append(item.attrib)
    #     meta['Subject Terms'].append(item.text)

    for item in root.iter('PubFrosting'):
        for title in item.iter('Title'):
            meta['Title of Newspaper'].append(item[2].text)
            break
    
    for item in root.iter('PubFrosting'):
        for title in root.iter('Subjects'):
            meta['Subjects'].append(title[0].text)

    for item in root.iter('TextInfo'):
        for title in item:
            meta['Word Count'].append(title.attrib['WordCount'])
    # df_meta = pd.DataFrame(data=meta)

    for item in root.iter('PubFrosting'):
        for title in root.iter('MpubId'):
            meta['MpubId'].append(title.text)

    for item in root.iter('PubFrosting'):
        for title in root.iter('StartDate'):
            meta['Start Date'].append(title.text)
            break

    for item in root.iter('PubFrosting'):
        for title in root.iter('EndDate'):
            meta['End Date'].append(title.text)
            break
            
    # add na if any header info is not found
    for k in meta:
        if not meta[k]:
            meta[k].append('n/a')

    return meta

path = '/home/ec2-user/SageMaker/data/Global_Newsstream_2013_bernanke/1442154774.xml'
meta = extract_meta(path)

meta_df = pd.DataFrame(data=meta)
meta_df.head()

,Keyword,GOID,Title,Contributors,Contributor FirstName,Contributor LastName,Numeric Date,Start Date,End Date,Qualifier,...,StartPage,DocSection,ColumnHeader,DocEdition,Title Keywords,Terms,Word Count,Title of Newspaper,Subjects,MpubId
0,n/a,1442154774,Many gains were in low-pay jobs,n/a,n/a,n/a,2013-07-06,2013-07-06,2013-07-06,"Buffalo, N.Y.",...,B.15,Financial,n/a,n/a,n/a,"[Terms, {}, , [GenSubjTerm, {'TermSource': 'ABI', 'PndxHeader': 'true'}, , GenSubjValue, {}, Investment policy], [GenSubjTerm, {'TermSource': 'ABI', 'PndxHeader': 'true'}, , GenSubjValue, {}, Health care industry], [GenSubjTerm, {'TermSource': 'ABI', 'PndxHeader': 'true'}, , GenSubjValue, {}, Government spending], [GenSubjTerm, {'TermSource': 'ABI', 'PndxHeader': 'true'}, , GenSubjValue, {}, Hotels & motels], [GenSubjTerm, {'TermSource': 'ABI', 'PndxHeader': 'true'}, , GenSubjValue, {}, Restaurants], [GenSubjTerm, {'TermSource': 'ABI', 'PndxHeader': 'true'}, , GenSubjValue, {}, Social security taxes], [Term, {'TermSource': 'ABI', 'PndxHeader': 'true'}, , Geographic, {}, United States--US]]",n/a,Buffalo News,General Interest Periodicals--United States,45728


In [4]:
for title in root.iter('CompanyTerm'):
    print(title.tag, title.attrib, title.text)
    for item in title:
        print(item.tag, item.attrib)

NameError: name 'root' is not defined

In [5]:
meta

{'Keyword': ['n/a'],
 'GOID': ['1442154774'],
 'Title': ['Many gains were in low-pay jobs'],
 'Contributors': ['n/a'],
 'Contributor FirstName': ['n/a'],
 'Contributor LastName': ['n/a'],
 'Numeric Date': ['2013-07-06'],
 'Start Date': ['2013-07-06'],
 'End Date': ['2013-07-06'],
 'Qualifier': ['Buffalo, N.Y.'],
 'Publisher Country': ['United States'],
 'Language': ['English'],
 'Source': ['Newspapers'],
 'StartPage': ['B.15'],
 'DocSection': ['Financial'],
 'ColumnHeader': ['n/a'],
 'DocEdition': ['n/a'],
 'Title Keywords': ['n/a'],
 'Terms': [['Terms',
   {},
   '',
   ['GenSubjTerm',
    {'TermSource': 'ABI', 'PndxHeader': 'true'},
    '',
    'GenSubjValue',
    {},
    'Investment policy'],
   ['GenSubjTerm',
    {'TermSource': 'ABI', 'PndxHeader': 'true'},
    '',
    'GenSubjValue',
    {},
    'Health care industry'],
   ['GenSubjTerm',
    {'TermSource': 'ABI', 'PndxHeader': 'true'},
    '',
    'GenSubjValue',
    {},
    'Government spending'],
   ['GenSubjTerm',
    {'TermS

## 2. Textfield Parsing & Analysis

### 2.1 Text metadata

In [6]:
# Parse the xml by paragraph and sentence
def extract_paragraphs_new(path):
    tree=et.parse(path)
    root=tree.getroot()
    del_list = []
    full = ''
    for text in root.iter('Text'):
        full = text.text
    for i in range(len(full)):
        if full[i:i+3] == '<p>':
            del_list.append(i)
    temp = {'Paragraph': [], 'Sentence': [], 'Overall Sentence': [], 'Text': []}
    sentence_count = 1
    #temp = {'Paragraph': [], 'Text': []}
    for i in range(len(del_list)-1):
        text = full[(del_list[i]+3):(del_list[i+1])]
        text = text.replace("\n","")
        text = text.replace("<p>","")
        text = text.replace("</p>","")
        text = text.replace("<i>","")
        text = text.replace("</i>","")
        text = text.replace("<b>","")
        text = text.replace("</b>","")
        all_sen = tokenize.sent_tokenize(text)
        for j in range(len(all_sen)):
            temp['Paragraph'].append(i+1)
            temp['Sentence'].append(j+1)
            temp['Overall Sentence'].append(sentence_count)
            temp['Text'].append(all_sen[j])
            sentence_count += 1
    df = pd.DataFrame(data=temp)
    return df

### 2.2 Sentiment Analysis (spaCY)

In [8]:
# for sentiment analysis
from spacytextblob.spacytextblob import SpacyTextBlob
import nltk
import spacy
nlp = spacy.load('/home/ec2-user/SageMaker/Getting Started/2022.05.25/Resources/Models/en_core_web_sm-3.2.0/en_core_web_sm/en_core_web_sm-3.2.0')
nlp.add_pipe('spacytextblob')

/home/ec2-user/SageMaker/.conda/envs/textblob/lib/python3.10/site-packages/spacy/util.py:865: UserWarning: [W095] Model 'en_core_web_sm' (3.2.0) was trained with spaCy v3.2 and may not be 100% compatible with the current version (3.4.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [9]:
def get_tags(sentence):
    """Return POS tags of each sentence."""
    res = []
    for token in nlp(sentence):
        res.append((token.text, token.pos_, token.dep_))
    return res

In [10]:
def get_polarity(sentence):
    """Return polarity score of each sentence."""
    token = nlp(sentence)
    polarity = token._.blob.polarity
    return polarity

In [11]:
def get_subjectivity(sentence):
    """Return subjectivity score of each sentence."""
    token = nlp(sentence)
    subjectivity = token._.blob.subjectivity
    return subjectivity

In [12]:
def extract_tokens_plus_meta(doc:spacy.tokens.doc.Doc):
    """Extract tokens and metadata from individual spaCy doc."""
    return [
        (i.text, i.i, i.lemma_, i.ent_type_, i.tag_, 
         i.dep_, i.pos_, i.is_stop, i.is_alpha, 
         i.is_digit, i.is_punct) for i in doc
    ]

def tidy_tokens(docs):
    """Extract tokens and metadata from list of spaCy docs."""
    
    cols = [
        "doc_id", "token", "token_order", "lemma", 
        "ent_type", "tag", "dep", "pos", "is_stop", 
        "is_alpha", "is_digit", "is_punct"
    ]
    
    meta_df = []
    for ix, doc in enumerate(docs):
        meta = extract_tokens_plus_meta(doc)
        meta = pd.DataFrame(meta)
        meta.columns = cols[1:]
        meta = meta.assign(doc_id = ix).loc[:, cols]
        meta_df.append(meta)
        
    return pd.concat(meta_df) 

I added a table to hold all the GOIDs and paths of the XMLs. I also rewrote the keyword search function so that each keyword could be searched in all the XMLs at once.

In [24]:
xml_info = pd.read_csv('xml_path.csv')
all_path = xml_info['Path'].tolist()
all_id = xml_info['GOID'].tolist()

In [25]:
def keyword_search(all_id, all_path, keyword):
    temp = pd.DataFrame()
    for i in range(len(all_path)):
        extracts = extract_paragraphs_new(all_path[i])
        extracts['GOID'] = all_id[i]
        extracts['Polarity'] = extracts['Text'].apply(get_polarity)
        extracts['Subjectivity'] = extracts['Text'].apply(get_subjectivity)
        temp = temp.append(extracts)
    print(temp['Text'])
    result = temp[temp['Text'].contains(keyword)]
    return result        

In [26]:
keyword_search(all_id, all_path, 'Yellen')

0     THE US lurched towards a so-called "fiscal cliff" with serious economic consequences today, despite signs Senate leaders were close to reaching a deal on extending tax cuts that offered a partial settlement of the budget crisis.                                                                                                                         
1     Unable to negotiate a broader budget agreement, Republican and Democrat leaders focused overnight on a compromise to prevent automatic tax increases for all except a small percentage of top income earners.                                                                                                                                                
2     This attempt at a partial fix left unclear how to prevent huge automatic government spending cuts, also due to take effect after today's fiscal-cliff deadline, which would have a negative impact on the US economy unless they could be suspended or delayed quickly.                   

/tmp/ipykernel_16623/4195683764.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(extracts)


AttributeError: 'Series' object has no attribute 'contains'

In [27]:
def get_number(all_id, all_path):
    temp = {'GOID': all_id, 'Paragraph Number': [], 'Sentence Number': []}
    for i in range(len(all_path)):
        extracts = extract_paragraphs_new(all_path[i])
        temp['Paragraph Number'].append(extracts['Paragraph'].max())
        temp['Sentence Number'].append(extracts['Overall Sentence'].max())
    df = pd.DataFrame(data=temp)   
    return df

In [28]:
get_number(all_id, all_path)

,GOID,Paragraph Number,Sentence Number
0,2123327034,47,62


In [30]:
extract_paragraphs_new('/home/ec2-user/SageMaker/data/Global_Newsstream_2013_bernanke/1265819747.xml')

,Paragraph,Sentence,Overall Sentence,Text
0,1,1,1,"THE US lurched towards a so-called ""fiscal cliff"" with serious economic consequences today, despite signs Senate leaders were close to reaching a deal on extending tax cuts that offered a partial settlement of the budget crisis."
1,2,1,2,"Unable to negotiate a broader budget agreement, Republican and Democrat leaders focused overnight on a compromise to prevent automatic tax increases for all except a small percentage of top income earners."
2,3,1,3,"This attempt at a partial fix left unclear how to prevent huge automatic government spending cuts, also due to take effect after today's fiscal-cliff deadline, which would have a negative impact on the US economy unless they could be suspended or delayed quickly."
3,4,1,4,"Talks involving Democrat Senate majority leader Harry Reid and his Republican counterpart, minority leader Mitch McConnell, stalled yesterday over a series of disputed side issues including estate tax rises and the formula for increasing the social security payments of retirees."
4,5,1,5,"But Senator McConnell appeared to have veered away from a blunt refusal to accept any form of tax increases -- even if he and most other Republicans still baulked at the original proposal of President Barack Obama and his Democratic Party that taxes go up for people earning more than $US250,000 ($240,000) a year."
5,6,1,6,"According to one report on the negotiations, the two sides were very close: Democrats had shown a willingness in private talks to lift their income tax threshold to $US450,000 for couples, while Republicans were prepared to accept a compromise of about $US550,000."
6,7,1,7,An automatic $US532 billion in tax increases and $US136bn in spending reductions will take effect from a deadline of midnight Washington time -- 4pm AEDT today -- in the event no legislated deal is reached beforehand.
7,8,1,8,"The deadline for these measures -- dubbed the fiscal cliff by Federal Reserve chairman Ben Bernanke because of their potentially disastrous economic impact -- was set by congress more than a year ago, during talks on raising the government's debt limit, as an incentive for both sides to get serious about doing a deal on deficit and debt reduction."
8,9,1,9,"Instead, Republicans and Democrats have remained divided, sticking with their diametrically opposed positions on tax increases and cuts to government programs right up to the deadline."
9,10,1,10,"Economists estimate the full brunt of the fiscal cliff, if allowed to happen unchecked, would be a 0.5 per cent cut to growth next year, potentially pushing the US into another recession and causing global reverberations."


In [32]:
extracts_2123327034 = extract_paragraphs_new('/home/ec2-user/SageMaker/data/Global_Newsstream_2013_bernanke/1265819747.xml')
#extracts_2123327034['Tag'] = extracts_2123327034['Text'].apply(get_tags)
extracts_2123327034['Polarity'] = extracts_2123327034['Text'].apply(get_polarity)
extracts_2123327034['Subjectivity'] = extracts_2123327034['Text'].apply(get_subjectivity)
#extracts_2123327034.to_csv('extracts_2123327034.csv', index=False)

In [33]:
docs_2123327034 = list(nlp.pipe(extracts_2123327034['Text']))
nlp_info_2123327034 = tidy_tokens(docs_2123327034)
#nlp_info_2123327034.to_csv('nlp_info_2123327034.csv', index=False)

In [35]:
# Newspaper
extracts_1545387815 = extract_paragraphs_new('/home/ec2-user/SageMaker/data/Global_Newsstream_2013_bernanke/1265819747.xml')
#extracts_1545387815['Tag'] = extracts_1545387815['Text'].apply(get_tags)
extracts_1545387815['Polarity'] = extracts_1545387815['Text'].apply(get_polarity)
extracts_1545387815['Subjectivity'] = extracts_1545387815['Text'].apply(get_subjectivity)
#extracts_1545387815.to_csv('extracts_1545387815.csv', index=False)

In [36]:
yellen_1545387815 = extracts_1545387815[extracts_1545387815['Text'].str.contains('Yellen')]

In [37]:
extracts_1545387815[extracts_1545387815['Text'].str.contains('Wall Street')]

,Paragraph,Sentence,Overall Sentence,Text,Polarity,Subjectivity


In [39]:
docs_1545387815 = list(nlp.pipe(extracts_1545387815['Text']))
nlp_info_1545387815 = tidy_tokens(docs_1545387815)
nlp_info_1545387815.to_csv('nlp_info_1545387815.csv', index=False)

In [41]:
# Newspaper
extracts_1545553533 = extract_paragraphs_new('/home/ec2-user/SageMaker/data/Global_Newsstream_2013_bernanke/1265819747.xml')
#extracts_1545553533['Tag'] = extracts_1545553533['Text'].apply(get_tags)
extracts_1545553533['Polarity'] = extracts_1545553533['Text'].apply(get_polarity)
extracts_1545553533['Subjectivity'] = extracts_1545553533['Text'].apply(get_subjectivity)

In [42]:
yellen_1545553533 = extracts_1545553533[extracts_1545553533['Text'].str.contains('Yellen')]

In [43]:
extracts_1545553533[extracts_1545553533['Text'].str.contains('Wall Street')]

,Paragraph,Sentence,Overall Sentence,Text,Polarity,Subjectivity


In [44]:
docs_1545553533 = list(nlp.pipe(extracts_1545553533['Text']))
nlp_info_1545553533 = tidy_tokens(docs_1545553533)
nlp_info_1545553533.to_csv('nlp_info_1545553533.csv', index=False)

In [45]:
# Newspaper
extracts_1545554849 = extract_paragraphs_new('/home/ec2-user/SageMaker/data/Global_Newsstream_2013_bernanke/1265819747.xml')
#extracts_1545554849['Tag'] = extracts_1545554849['Text'].apply(get_tags)
extracts_1545554849['Polarity'] = extracts_1545554849['Text'].apply(get_polarity)
extracts_1545554849['Subjectivity'] = extracts_1545554849['Text'].apply(get_subjectivity)

In [46]:
yellen_1545554849 = extracts_1545554849[extracts_1545554849['Text'].str.contains('Yellen')]

In [47]:
extracts_1545554849[extracts_1545554849['Text'].str.contains('Wall Street')]

,Paragraph,Sentence,Overall Sentence,Text,Polarity,Subjectivity


In [48]:
docs_1545554849 = list(nlp.pipe(extracts_1545554849['Text']))
nlp_info_1545554849 = tidy_tokens(docs_1545554849)
nlp_info_1545554849.to_csv('nlp_info_1545554849.csv', index=False)

In [49]:
# News
extracts_1550716133 = extract_paragraphs_new('/home/ec2-user/SageMaker/data/Global_Newsstream_2013_bernanke/1265819747.xml')
#extracts_1550716133['Tag'] = extracts_1550716133['Text'].apply(get_tags)
extracts_1550716133['Polarity'] = extracts_1550716133['Text'].apply(get_polarity)
extracts_1550716133['Subjectivity'] = extracts_1550716133['Text'].apply(get_subjectivity)

In [50]:
yellen_1550716133 = extracts_1550716133[extracts_1550716133['Text'].str.contains('Yellen')]

In [51]:
extracts_1550716133[extracts_1550716133['Text'].str.contains('Wall Street')]

,Paragraph,Sentence,Overall Sentence,Text,Polarity,Subjectivity


In [52]:
docs_1550716133 = list(nlp.pipe(extracts_1550716133['Text']))
nlp_info_1550716133 = tidy_tokens(docs_1550716133)
nlp_info_1550716133.to_csv('nlp_info_1550716133.csv', index=False)

In [53]:
# Web
extracts_2046284852 = extract_paragraphs_new('/home/ec2-user/SageMaker/data/Global_Newsstream_2013_bernanke/1265819747.xml')
#extracts_2046284852['Tag'] = extracts_2046284852['Text'].apply(get_tags)
extracts_2046284852['Polarity'] = extracts_2046284852['Text'].apply(get_polarity)
extracts_2046284852['Subjectivity'] = extracts_2046284852['Text'].apply(get_subjectivity)

In [54]:
yellen_2046284852 = extracts_2046284852[extracts_2046284852['Text'].str.contains('Yellen')]

In [55]:
# keyword "Wall Street"
extracts_2046284852[extracts_2046284852['Text'].str.contains('Wall Street')]

,Paragraph,Sentence,Overall Sentence,Text,Polarity,Subjectivity


In [56]:
docs_2046284852 = list(nlp.pipe(extracts_2046284852['Text']))
nlp_info_2046284852 = tidy_tokens(docs_2046284852)
nlp_info_2046284852.to_csv('nlp_info_2046284852.csv', index=False)

In [57]:
# Concatenate all the result tables
yellen = pd.concat([yellen_1545387815, yellen_1545553533, yellen_1545554849, yellen_1550716133, yellen_2046284852], 
                   keys = [1545387815, 1545553533, 1545554849, 1550716133, 2046284852]).reset_index()

In [58]:
yellen = yellen.rename(columns={'level_0': 'GOID'})

In [59]:
yellen = yellen.drop(columns=['level_1'])

In [60]:
yellen.to_csv('yellen.csv', index=False)